## Messari standardised subgraphs

https://github.com/messari/subgraphs

### Solarbeam - https://docs.solarbeam.io/faq

12/09/22



In [10]:
import pandas as pd
import numpy as np
import time
import requests
import datetime as dt

In [11]:
endpoint = 'https://api.thegraph.com/subgraphs/name/messari/solarbeam-moonriver'

In [12]:
#Check if subgraph is up to date
query = """{
  _meta{
    block{
      timestamp
  		hash
      number
    }
    hasIndexingErrors
    deployment
  }
}"""
request = requests.post(endpoint, json={'query':query})
data = request.json()
#Check if it is ok to run i.e. within 5 days of today
print(dt.datetime.fromtimestamp(int(data['data']['_meta']['block']['timestamp'], base = 0)))
int(data['data']['_meta']['block']['timestamp'], base = 0) > (dt.datetime.today().timestamp() - 3600*24*5)

2022-09-08 00:49:48


True

6-Month Average Daily Volume

In [13]:
query = """{
  financialsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    id
    protocol{
      name
    }
    dailyVolumeUSD
    dailyTotalRevenueUSD
    dailySupplySideRevenueUSD
    dailyProtocolSideRevenueUSD
    timestamp
    blockNumber
  }
}
"""

In [14]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [15]:
df = pd.json_normalize(data['data']['financialsDailySnapshots'])

In [16]:
df['dailyVolumeUSD'].astype(float).mean()

1688743.657271109

6-Month Average TVL

In [17]:
#DefiLlama https://defillama.com/protocols/dexes 
lma = pd.read_csv("https://api.llama.fi/dataset/solarbeam.csv")

In [18]:
lma

,Unnamed: 0,Date,Timestamp,Solarbeam,Solarbeam.1,Solarbeam.2,Solarbeam.3,Solarbeam.4,Solarbeam.5,Solarbeam.6,...,Solarbeam.368,Solarbeam.369,Solarbeam.370,Solarbeam.371,Solarbeam.372,Solarbeam.373,Solarbeam.374,Solarbeam.375,Solarbeam.376,Solarbeam.377
0,NaN,NaN,NaN,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,...,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes
1,NaN,NaN,NaN,Total,moonriver,Total,Total,Total,Total,Total,...,moonriver,moonriver,moonriver,moonriver,moonriver,moonriver,moonriver,moonriver,moonriver,moonriver
2,NaN,NaN,NaN,TVL,TVL,Tokens(USD),Tokens(USD),Tokens(USD),Tokens(USD),Tokens(USD),...,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens
3,NaN,NaN,NaN,NaN,NaN,USDT,moonriver,kusama,frax,tether,...,UNKNOWN (moonriver:0xd850e17bdb1b71fbdbc7e8b4e...,UNKNOWN (moonriver:0xf4c6850b6e3288e81be542909...,UNKNOWN (moonriver:0xe413a631e8a9a10958d9b7c64...,KOGECOIN,LINK,WMOVR,FRAX,BNB,ETH,MIMATIC
4,NaN,04/09/2021,1.630714e+09,324457.32223,324457.32223,324457.32223,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,NaN,08/09/2022,1.662595e+09,22247664.164635815,22247664.164635815,NaN,NaN,159592.24232025963,3943353.1956471996,1670503.555107,...,NaN,NaN,NaN,NaN,0.000714077317772054,681117.9547586939,225.0888047310958,42.227312056955185,774.0569245674743,42.23896550040043
358,NaN,09/09/2022,1.662682e+09,22476911.652070314,22476911.652070314,NaN,NaN,166090.4008240792,3955247.970726666,1677775.9868319447,...,NaN,NaN,NaN,NaN,0.000714077317772054,682741.0194571925,225.0888047310958,41.68270735741619,775.8379540781376,42.23896550040043
359,NaN,10/09/2022,1.662768e+09,22957053.044496506,22957053.044496506,NaN,NaN,172529.83288902362,3994448.490452045,1679821.5752317996,...,NaN,NaN,NaN,NaN,0.000714077317772054,682242.0439518137,224.11690176926714,40.703133563533555,775.1882764238085,42.23896550040043
360,NaN,11/09/2022,1.662854e+09,23196344.2907035,23196344.2907035,NaN,NaN,167955.48744865018,3900556.99917124,1826287.6319157467,...,NaN,NaN,NaN,NaN,0.000714077317772054,667117.3900853866,223.13881749407574,40.78914100251957,778.5040532794663,42.23896550040043


In [19]:
lma.iloc[:,3]

0                   Dexes
1                   Total
2                     TVL
3                     NaN
4            324457.32223
              ...        
357    22247664.164635815
358    22476911.652070314
359    22957053.044496506
360      23196344.2907035
361     22875220.06792463
Name: Solarbeam, Length: 362, dtype: object

In [20]:
#Just eth currently can take total by changing the header
lma.iloc[-183:, 3].astype(float).mean()

23941262.584223118

Largest Pool Concentration

In [21]:
df = pd.DataFrame()
ts = 0

while True:
    sl = len(df)
    query = """
{
  liquidityPools(first:1000, orderBy: createdTimestamp, orderDirection: asc, where: {createdTimestamp_gt: """+f'"{ts}"'+"""}){
    id
    name
    totalValueLockedUSD
    cumulativeVolumeUSD
    createdTimestamp
    createdBlockNumber
  }
}
"""
    request = requests.post(endpoint, json={'query':query})
    data = request.json()

    pool = pd.json_normalize(data['data']['liquidityPools'])
    df = df.append(pool)
    df.drop_duplicates(inplace=True)
    if len(df) == sl:
      break
    ts = df['createdTimestamp'].max()


/tmp/ipykernel_68602/2291472997.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)


In [22]:
df

,id,name,totalValueLockedUSD,cumulativeVolumeUSD,createdTimestamp,createdBlockNumber
0,0x7eda899b3522683636746a2f3a7814e6ffca75e1,Solarbeam SolarBeam Token/Wrapped MOVR,741522.9429846527173133529607139964,206194505.7851295096519765300093201,1630515054,442666
1,0x0acdb54e610dabc82b8fa454b21ad425ae460df9,Solarbeam Wrapped MOVR/RiverBoat,13316.57275797823539475272901143798,9614635.845514196959256796631780624,1630611180,450311
2,0xf9b7495b833804e4d894fc5f7b39c10016e0a911,Solarbeam SolarBeam Token/RiverBoat,426.5500756120380913860188597933458,0,1630611240,450316
3,0xe537f70a8b62204832b8ba91940b77d3f79aeb81,Solarbeam Wrapped MOVR/USD Coin,4061857.989138,1019211461.647698256316232009557304,1630679850,455797
4,0xdb66be1005f5fe1d2f486e75ce3c50b52535f886,Solarbeam SolarBeam Token/USD Coin,173083.1426018863748999529839183593,175019033.402007,1630680156,455822
...,...,...,...,...,...,...
601,0x66aebb83abfc0cb23c57f736095cf3ee804eca7c,Solarbeam Frax/Bitcoin Trade,1.557650777082093404491059762498272,0,1661677380,2458635
602,0x7a245c4ed3aa3b4b11a1920539a376b3f8d2e927,Solarbeam Ethereum/Bitcoin Trade,1.888169221959205906227710198617098,0,1661678448,2458715
603,0x9e544c42cdce25a4196dfe29c8da8a5ef0ced726,Solarbeam Wrapped MOVR/TOKEN1,0.1177419230891923580201050147562901,0,1662288648,2502406
604,0x19b94f0b93d760798029bd0191359aef41ba7f8f,Solarbeam Wrapped MOVR/TOKEN2,0.1238436711562982252098972177934396,0,1662288726,2502411


In [23]:
df['totalValueLockedUSD'].astype(float).max()/df['totalValueLockedUSD'].astype(float).sum()

0.25173650947113557

In [24]:
df['totalValueLockedUSD'].astype(float).nlargest(10).sum()/df['totalValueLockedUSD'].astype(float).sum()

0.9538537747023763

Liquidity Pools with > $1mn TVL

In [25]:
len(df.loc[df['totalValueLockedUSD'].astype(float) > 1000000])

5

Monthly Active Users

In [26]:
query = """{
  usageMetricsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    timestamp
    blockNumber
    id
    dailyActiveUsers
    dailySwapCount
    dailyDepositCount
    dailyWithdrawCount 
  }
}
"""

In [27]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [28]:
df = pd.json_normalize(data['data']['usageMetricsDailySnapshots'])

In [29]:
df['dailyActiveUsers'].astype(float).mean()

750.8907103825137

Implied slippage

In [30]:
query = """
{
  liquidityPools(first:10, orderBy: totalValueLockedUSD, orderDirection: desc){
    id
    name
    totalValueLockedUSD
    cumulativeVolumeUSD
    createdTimestamp
    createdBlockNumber
    inputTokens{
      id
      name
      symbol
      lastPriceUSD
      decimals
    }
    inputTokenBalances
  }
}
"""
request = requests.post(endpoint, json={'query':query})
if request.ok:
    data = request.json()

In [31]:
data

{'data': {'liquidityPools': [{'id': '0xe537f70a8b62204832b8ba91940b77d3f79aeb81',
    'name': 'Solarbeam Wrapped MOVR/USD Coin',
    'totalValueLockedUSD': '4061857.989138',
    'cumulativeVolumeUSD': '1019211461.647698256316232009557304',
    'createdTimestamp': '1630679850',
    'createdBlockNumber': '455797',
    'inputTokens': [{'id': '0x98878b06940ae243284ca214f92bb71a2b032b8a',
      'name': 'Wrapped MOVR',
      'symbol': 'WMOVR',
      'lastPriceUSD': '12.19711299744073263310725607952758',
      'decimals': 18},
     {'id': '0xe3f5a90f9cb311505cd691a46596599aa1a0ad7d',
      'name': 'USD Coin',
      'symbol': 'USDC',
      'lastPriceUSD': '1',
      'decimals': 6}],
    'inputTokenBalances': ['166508992332459413171843', '2030928994569']},
   {'id': '0xea3d1e9e69addfa1ee5bbb89778decd862f1f7c5',
    'name': 'Solarbeam Wrapped MOVR/xcKSM',
    'totalValueLockedUSD': '3331382.340131707578613537971902882',
    'cumulativeVolumeUSD': '27950052.85977082675136835631504236',
    'creat

In [32]:
slip1000 = []
slip10000 = []
slip100000 = []

for pool in data['data']['liquidityPools']:
    token0 = pool['inputTokens'][0]
    token0_reserveUSD = (float(pool['inputTokenBalances'][0])/10**token0['decimals'])*float(token0['lastPriceUSD'])
    
    token1 = pool['inputTokens'][1]
    token1_reserveUSD = (float(pool['inputTokenBalances'][1])/10**token1['decimals'])*float(token1['lastPriceUSD'])

    try:
        pi1000 = 1000/token0_reserveUSD
    except:
        pi1000 = 1000/token1_reserveUSD
    
    try:
        pi10000 = 10000/token0_reserveUSD
    except:
        pi10000 = 10000/token1_reserveUSD
    
    try:
        pi100000 = 100000/token0_reserveUSD
    except:
        pi100000 = 100000/token1_reserveUSD
    
    slip1000.append(pi1000)
    slip10000.append(pi10000)
    slip100000.append(pi100000)


In [33]:
np.mean(slip1000)

0.007911287314104076

In [34]:
np.mean(slip10000)

0.07911287314104073

In [35]:
np.mean(slip100000)

0.7911287314104074